#### BayesianOptimizer를 이용한 CNN모델 하이퍼파라메터 튜닝
* tensorflow를 사용하기 때문에 `t212p39` anaconda env 사용

#### 라이브러리 불러오기

In [25]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

from datetime import datetime as dt

In [3]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [4]:
train_x, test_x = train_x/255.0, test_x/255.0

In [6]:
train_x, test_x = tf.expand_dims(train_x, -1), tf.expand_dims(test_x, -1)

In [7]:
train_y, test_y = to_categorical(train_y, 10), to_categorical(test_y, 10)

In [8]:
def create_cnn_model(filters, kernel_size, pool_size, dense_units):
    model = Sequential([
        Conv2D(
            filters, kernel_size, activation='relu', 
            input_shape=(train_x.shape[1:])
        ),
        MaxPooling2D(pool_size),
        Flatten(),
        Dense(dense_units, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [16]:
pbounds = {
    'filters': (16, 256),
    'kernel_size': (3, 5),
    'pool_size': (2, 3),
    'dense_units': (32, 512),
    'epochs': (5, 20)
}

In [17]:
def target_func(filters, kernel_size, pool_size, dense_units, epochs):
    model = create_cnn_model(
        int(filters), int(kernel_size), int(pool_size), int(dense_units)
    )
    model.fit(train_x, train_y, epochs=int(epochs), verbose=0)
    _, accuracy = model.evaluate(test_x, test_y, verbose=0)
    return accuracy

In [18]:
optimizer = BayesianOptimization(
    f=target_func,
    pbounds=pbounds,
    verbose=2,
    random_state=123
)

In [19]:
_start = dt.now()
optimizer.maximize(init_points=2, n_iter=10)
print(f'처리시간: {dt.now()-_start}')

|   iter    |  target   | dense_... |  epochs   |  filters  | kernel... | pool_size |
-------------------------------------------------------------------------------------
| 1         | 0.9871    | 366.3     | 9.292     | 70.44     | 4.103     | 2.719     |


KeyboardInterrupt: 

In [ ]:
print("Best parameters:", optimizer.max['params'])
print("Best validation accuracy:", optimizer.max['target'])

#### 제안-평가-등록

In [20]:
optimizer = BayesianOptimization(
    f=None,
    pbounds=pbounds,
    verbose=2,
    random_state=123
)

In [22]:
utility = UtilityFunction(
    kind='ucb', # 탐색과 활용 사이의 균형 유지, 불확실한 지점을 더 많이 탐색
    kappa=2.5,  # UCB전략에서 얼마의 불확실성을 고려할지 지정
                # 값이 높으면 탐색을, 값이 낮으면 활용을 강조함
    xi=0.0      # 탐색전략에서 사용되는 파라미터로 얼마나 큰 개선을 고려할지 조정
                # 값이 높으면 큰 개선을 값이 낮으면 작은 개선을 탐색함
)

In [23]:
next_point = optimizer.suggest(utility)
print(next_point)

{'dense_units': 366.3052090869736, 'epochs': 9.292090024255693, 'filters': 70.44434885540875, 'kernel_size': 4.1026295381657825, 'pool_size': 2.719468969785563}


In [26]:
_start = dt.now()
target = target_func(**next_point)
print(target)
print(f'처리시간: {dt.now()-_start}')

0.9879999756813049
처리시간: 0:19:08.858003


In [27]:
optimizer.register(
    params=next_point,
    target=target
)

In [28]:
_start = dt.now()
for epoch in range(10):
    __start = dt.now()
    next_point = optimizer.suggest(utility)
    target = target_func(**next_point)
    optimizer.register(params=next_point, target=target)
    print(f'{epoch}: {target} - {next_point}, time: {dt.now()-__start}')
    
print(optimizer.max)
print(f'처리시간: {dt.now()-_start}')

KeyboardInterrupt: 